In [1]:
### MOUNT DRIVE .....

In [2]:
## First Import statements
import numpy as np
import pandas as pd
from IPython.display import display
import pickle

from scipy import stats
from xml.dom import minidom

from sklearn.metrics import pairwise_distances, accuracy_score


# !pip3 install tqdm
from tqdm import tqdm
import operator
from functools import reduce

In [3]:
FOLDER_TRAIN = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Data/Training/'
# FILE_TOPICS = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Data/topics.txt'
FILE_TOPICS = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Data/topics-all.txt'

In [4]:
# !ls "$FOLDER_TRAIN"

In [5]:
## Set random seed
RANDOM_STATE = 22
np.random.seed(RANDOM_STATE)

In [6]:
## Read data
topic_remove = '3d_Printer' ## Remove 3D-printer
list_doc_types = []
with open(FILE_TOPICS, 'r') as fp:
    topic = fp.readline()
    while topic:
        topic = topic.replace("\n", "")
        list_doc_types.append(topic)
        topic = fp.readline()

list_doc_types.remove(topic_remove)
print(list_doc_types)

['Astronomy', 'Coffee', 'Space', 'Anime', 'Biology', 'Cooking', 'Windows_Phone', 'Arduino', 'Chess', 'Law', 'Wood_Working']


In [7]:
print(list_doc_types)

['Astronomy', 'Coffee', 'Space', 'Anime', 'Biology', 'Cooking', 'Windows_Phone', 'Arduino', 'Chess', 'Law', 'Wood_Working']


In [8]:
def get_train_val_test_data(file_name):
    xmldoc = minidom.parse(file_name)
    xml_list = xmldoc.getElementsByTagName('row') ## tag using 'row'
    
    # print(f"Inside get_train_val_test_data(), len(item_list) = {len(item_list)}")

    item_list = [x.attributes['Body'].value for x in xml_list]

    train_list = item_list[0:500] ## first 500 train
    val_list =  item_list[500:700] ## next 200 val
    test_list = item_list[700:1200] ## next 500 test

    ## delete original list.
    del item_list
    del xmldoc

    ## return the new lists
    return train_list, val_list, test_list

In [9]:
def add_to_dataframe(df_old, to_add):
    df_old = df_old.append(pd.Series(to_add, index=df_old.columns), ignore_index=True)
    return df_old

In [10]:
## Create three dataframes.

## https://www.kite.com/python/answers/how-to-create-an-empty-dataframe-with-column-names-in-python
column_names =["content", "Label"]

df_train = pd.DataFrame(columns = column_names)
df_val = pd.DataFrame(columns = column_names)
df_test = pd.DataFrame(columns = column_names)

In [11]:
### Populate all topics
def populate_data_frames(df_train, df_val, df_test, list_doc_types):
    for topic in list_doc_types: ## iterating per topic/label
        label = topic ## assign label
        
        ## read using xml package
        file_name = FOLDER_TRAIN + topic + ".xml"
        xmldoc = minidom.parse(file_name)
        xml_list = xmldoc.getElementsByTagName('row') ## tag using 'row'

        ## get train, val, test lists
        train_list, val_list, test_list = get_train_val_test_data(file_name=file_name)

        print(len(train_list), len(val_list), len(test_list))

        ## add to dataframe
        for v1 in train_list:
            df_train = add_to_dataframe(df_old=df_train, to_add=[v1, label])
        for v2 in val_list:
            df_val = add_to_dataframe(df_old=df_val, to_add=[v2, label])
        for v3 in test_list:
            df_test = add_to_dataframe(df_old=df_test, to_add=[v3, label])


        ## delete original list
        del train_list
        del val_list
        del test_list

    ## return dataframes
    return df_train, df_val, df_test

### Call the function
df_train, df_val, df_test = populate_data_frames(df_train=df_train, df_val=df_val, df_test=df_test, list_doc_types=list_doc_types)

print(f"len df_train = {len(df_train)}")
print(f"len df_val = {len(df_val)}")
print(f"len df_test = {len(df_test)}")

500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
len df_train = 5500
len df_val = 2200
len df_test = 5500


## Shuffle dataframes

In [12]:
df_train = df_train.sample(frac=1, random_state=RANDOM_STATE)
df_val = df_val.sample(frac=1, random_state=RANDOM_STATE)

## Import libraries

In [13]:
## https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
## https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
## from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [14]:
## https://stackoverflow.com/questions/26693736/nltk-and-stopwords-fail-lookuperror
## https://stackoverflow.com/questions/26570944/resource-utokenizers-punkt-english-pickle-not-found

# nltk.download()
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Preprocess each sentence

In [15]:
## https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
## First lemmatize, and then stem.

def pre_process_stem_sentence(sentence, stop_words, stemmer, lemmatizer):
    ## save in another variable
    text = sentence
    ## remove HTML tags [using soup]
    soup = BeautifulSoup(text)
    text = soup.get_text()
    
    ## remove <a href> type things
    soup = BeautifulSoup(text) ## create soup again.
    for a in soup.findAll('a'):
        a.replaceWithChildren()
    
    text = str(soup) ## reform text


    ## remove unicode.
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r'[-+]?\d+', '', text)
    text = re.sub(r'[^\x00-\x7F]',' ', text)

    ## remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"https\S+", "", text)
    ## text = re.sub(r"www\S+", "", text)

    ## to be safe, remove HTML tags again using regex
    #### https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44  
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    ## convert to small letters.
    text = text.lower()

    ## replace newlines, tabs -> SPACE
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")

    ## {COLON} make <a>:<b> become <a>[space]<b>
    # text = text.replace(": ", ":") # :[space] -> : 
    text = text.replace(":", " ")  # : -> [space]
    
    ## {HYPHEN}
    # text = text.replace("- ", "-")
    text = text.replace("-", " ")

    ## numbers removal    
    text = re.sub(r'[-+]?\d+', '', text)

    ## punctuations removal
    text = text.translate((str.maketrans('','',string.punctuation)))   


    ## [remove anything EXCEPT english letters]
    ## https://stackoverflow.com/questions/6323296/python-remove-anything-that-is-not-a-letter-or-number
    text = re.sub(  "[^a-z ]",              # Anything except 0..9, a..z and A..Z
                    "",                     # replaced with nothing
                    text)                   # in this string   

    ## remove space initially and finally.
    text = text.lstrip()

    ## make double spaces become one space.
    text = re.sub('\s+', ' ', text)

    ## remove stop words (English).
    # print(f"Before stopwords removal, text.len = {len(text)}")

    tokens = word_tokenize(text)
    stop_words_removed_words_list = [t for t in tokens if not t in stop_words]

    # print(f"After tokenize and stopwords, len stop_words_removed_words_list = {len(stop_words_removed_words_list)}")

    ## apply lemmatization.
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stop_words_removed_words_list]
    
    ## apply stemming.
    stemmed_words = [stemmer.stem(word) for word in lemmatized_words]

    ## return the final pre-processed list-of-words.
    return stemmed_words

################ Test on one sentence #######################

## Obtain once.
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

## Preprocess per element of dataframe to test.
text_to_process = df_train['content'].iloc[499]
print(text_to_process)

print("--"*90)

preprocessed_text = pre_process_stem_sentence(sentence=text_to_process, stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer)
print(preprocessed_text)

<h2>You never mentioned your tablet model... or Arduino model</h2>

<p>There are some tablets designed to work with USB flash drives and printers. This is called OTG (on the go.</p>

<p><strong>You need:</strong></p>

<ul>
<li>A tablet supporting OTG</li>
<li>An OTG adapter that fits into your tablets USB slot and is compatible</li>
<li><a href="https://play.google.com/store/apps/details?id=com.primavera.arduino.listener" rel="nofollow">The Arduino Uno Communicator App</a></li>
<li>Arduino Uno (or clone) [Note: It says that it works with Atmega16U2 or Atmega8U2 programmed as a USB-to-serial converter so I would assume that that would cover a few boards more than the Uno.)</li>
</ul>

<p>Another alternitive is to look into <a href="https://www.google.com/search?q=arduino+bluetooth&amp;qscrl=1&amp;tbm=isch&amp;imgil=WyPC4O05Xn4rhM%253A%253Bhttps%253A%252F%252Fencrypted-tbn2.gstatic.com%252Fimages%253Fq%253Dtbn%253AANd9GcSfTtWqIuN9YYdd90OO_tlxsvLUYhV_pl7NfaMhHDzSRq9N6W9rNA%253B1200%253B12

In [16]:
doc = df_val['content'].iloc[499]
print(doc)

print("--"*85)

preprocessed = pre_process_stem_sentence(sentence=doc, stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer)
print(len(preprocessed))
print(preprocessed)

<p>This sounds like it's more a matter of determining the original name / wording used to file the copyright. </p>

<p>Without knowing more of that language, you're left to the typical sleuthing options:</p>

<ul>
<li>names of the company owners / major shareholders as of 15-18 years ago. </li>
<li>other DBAs and holding companies of the company originally presumed to hold the copyright. </li>
<li>brute force search of all categorically-related copyrights in the time range. </li>
</ul>

<p>The advice I would really like to give you is knowledge of how IP attorneys filed video game copyrights in the time span of 1985-2000.  This could reveal any unexpected filing categories that could have been used as part of niche or experimental copyright strategy (at that time) for this kind of IP. </p>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
70
['sound', 'like', 'matte

In [17]:
## Preprocess for each sentence of train-dataframe.
df_train["stemmed_content"] = df_train.apply(lambda row : pre_process_stem_sentence(sentence=row['content'], stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer), axis=1)
df_val["stemmed_content"] = df_val.apply(lambda row : pre_process_stem_sentence(sentence=row['content'], stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer), axis=1)
df_test["stemmed_content"] = df_test.apply(lambda row : pre_process_stem_sentence(sentence=row['content'], stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer), axis=1)

# Form Vocabulary from dataframe_train

In [18]:
def print_first_n_keys_and_vals_dict(dictionary, n=5):
    print(f"Len dictionary = {len(dictionary)}, printing first {n} keys, vals")
    itr = 0
    for key in dictionary:
        print(key, dictionary[key])
        itr += 1
        if itr == n:
            break

In [19]:
### Also remove one-len words
dictionary_vocab = {} ## empty dict.
for doc in df_train['stemmed_content']:
    # print(f"len doc = {len(doc)}")
    for word in doc:
        if len(word) <= 1:
            continue

        len_currently = len(dictionary_vocab) ## add to len. [idx new]

        if word not in dictionary_vocab:
            dictionary_vocab[word] = (0, len_currently)
        else:
            (val, idx) = dictionary_vocab[word]
            dictionary_vocab[word] = (val + 1, idx) ## Maintain the same index.

print(f"len dictionary_vocab = {len(dictionary_vocab)}")

len dictionary_vocab = 17506


In [20]:
# file_name = "/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Vocab.txt"
# with open(file_name, 'w') as fw:
#     for voc in dictionary_vocab:
#         fw.write(voc)
#         fw.write("\n")

In [21]:
# ## https://www.kite.com/python/answers/how-to-save-a-dictionary-to-a-file-in-python
# vocab_file = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/vocab_dict_df_train.pkl'

# a_file = open(vocab_file, "wb")
# pickle.dump(dictionary_vocab, a_file)
# a_file.close()

# # a_file = open(vocab_file, "rb")
# # dictionary_vocab = pickle.load(a_file)
# # print(dictionary_vocab)

## Keep only those docs in train whose lengths are above 3 words i.e.
### length of stemmed content > 3

In [22]:
lens = [len(x) for x in df_train['stemmed_content']]
print(max(lens))
print(min(lens))

1381
0


In [23]:
print(f"Before removal, len df_train = {len(df_train)}")

MIN_WORD_COUNT_TO_REMOVE = 2
# print(df_train[df_train['stemmed_content'].str.len() <= MIN_WORD_COUNT_TO_REMOVE]["stemmed_content"])
idxToRemove = df_train[df_train['stemmed_content'].str.len() <= MIN_WORD_COUNT_TO_REMOVE].index
print(f"To remove num items = {len(idxToRemove)}")

labels_to_remove = df_train['Label'].iloc[idxToRemove].values
print(np.unique(labels_to_remove, return_counts=True)) ## Almost fairly distributed.


df_train.drop(idxToRemove , inplace=True) ### Removing

print(f"After removal, len df_train = {len(df_train)}")

Before removal, len df_train = 5500
To remove num items = 224
(array(['Anime', 'Arduino', 'Astronomy', 'Biology', 'Chess', 'Coffee',
       'Cooking', 'Law', 'Space', 'Windows_Phone', 'Wood_Working'],
      dtype=object), array([18, 15, 29, 19, 18, 27, 24, 21, 19, 14, 20]))
After removal, len df_train = 5276


## Create Hamming Distance Vectors by representing with 0/1


In [24]:
def form_hamming_vector(list_words, vocab_dict):
    vec = np.zeros(len(vocab_dict)) # +1 for unknown word.
    for word in list_words:
        if word not in vocab_dict: ## add 1 to unkown word index. [DO NOT]
            # vec[UNKNOWN_WORD_INDEX] = 1
            continue
        else: ## word is present in vocab, get index.
            (value, idx) = vocab_dict[word]
            vec[idx] = 1
    return vec

In [25]:
## Create hamming vectors for each col of dataframe
df_train["ham_vector"] = df_train.apply(lambda row : form_hamming_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_val["ham_vector"] = df_val.apply(lambda row : form_hamming_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_test["ham_vector"] = df_test.apply(lambda row : form_hamming_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)

## Create eucledian vectors by representing how many times

In [26]:
def form_eucledian_vector(list_words, vocab_dict):
    vec = np.zeros(len(vocab_dict)) # +1 for unknown word.
    
    ## Form a small dictionary to store each word count
    dict_local_vocab = {}
    for word in list_words:
        if word not in dict_local_vocab:
            dict_local_vocab[word] = 1
        else:
            dict_local_vocab[word] = dict_local_vocab[word] + 1

    # print(dict_local_vocab)

    # unknown_word_count = 1
    for word in dict_local_vocab:
        if word not in vocab_dict: ## add 1 to unkown word index.
            continue
            # vec[UNKNOWN_WORD_INDEX] = unknown_word_count
            # unknown_word_count += 1
        else: ## word is present in vocab, get index.
            (value, idx) = vocab_dict[word] 
            vec[idx] = dict_local_vocab[word] ## replace with value of this word instead of 1.

    del dict_local_vocab
    return vec

In [27]:
df_train["euc_vector"] = df_train.apply(lambda row : form_eucledian_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_val["euc_vector"] = df_val.apply(lambda row : form_eucledian_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_test["euc_vector"] = df_test.apply(lambda row : form_eucledian_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)

In [28]:
# FOLDER_DATASET = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Dataframes-Dataset'
# df_train.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Dataframes-Dataset/train.csv', index=False)
# df_val.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Dataframes-Dataset/val.csv', index=False)
# df_test.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Dataframes-Dataset/test.csv', index=False)

# Stack each of these vertically to form hamming and eucledian vectors.

In [29]:
def get_2D_vector(list_np_arr):
    vec_2D = np.zeros((len(list_np_arr), len(list_np_arr[0])))
    idx = 0
    for vec in list_np_arr:
        vec_2D[idx] = vec
        idx += 1
    return vec_2D

In [30]:
vocab_size = len(dictionary_vocab)
print(f"vocab_size = {vocab_size}")

num_documents = len(df_train)
print(f"num_documents = {num_documents}")

print(df_train.columns.values)

vocab_size = 17506
num_documents = 5276
['content' 'Label' 'stemmed_content' 'ham_vector' 'euc_vector']


In [31]:
hamming_vectors_2D = get_2D_vector(list_np_arr=df_train['ham_vector'].values)
eucledian_vectors_2D = get_2D_vector(list_np_arr=df_train['euc_vector'].values)
print(hamming_vectors_2D.shape, eucledian_vectors_2D.shape)

(5276, 17506) (5276, 17506)


## For now save these dataframes

In [32]:
# df_train.to_csv("/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/train.csv", index=False)
# df_val.to_csv("/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/val.csv", index=False)
# df_test.to_csv("/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/test.csv", index=False)

## Now, finally create TF-IDF and pickel dump everything.

In [33]:
def compute_TF_IDF_all_train_set(hamming_vectors_2D, eucledian_vectors_2D, alpha=0.0001, beta=0.0001):
    num_documents, num_words = eucledian_vectors_2D.shape
    print(f"num_documents, num_words = {num_documents, num_words}")

    TF = np.zeros((num_documents, num_words))

    IDF = np.zeros((1, num_words))

    ## Calculate TF(d, w) = N(d, w)/W(d) ; where N(d, w): count(w) in document d , W(d): Total #words in document d
    for itr in range(num_documents): ## iterate row-wise
        doc_eucledian = eucledian_vectors_2D[itr]
        total_num_words_doc_eucledian = num_words - np.sum(doc_eucledian == 0)        
        TF[itr] = doc_eucledian/total_num_words_doc_eucledian

        # if itr == 1:
            # print(f"itr = {itr}, doc_euc[itr] = {np.unique(doc_eucledian[itr], return_counts=True)}")
            # print(f"itr = {itr}, TF[itr] = {np.unique(TF[itr], return_counts=True)}")
            # break

    ## Calculate IDF(d, w) = log( (D + alpha)/(C(w) + beta) ) ; C(w) -> Total # docs with word 'w' ; D -> Total # documents
    D = num_documents
    for itr in range(num_words): ## itereate col-wise
        C_w = np.sum(hamming_vectors_2D[:, itr] == 1) ## first calculate C(w)
        
        IDF[:, itr] = np.log( (D + alpha) / (C_w + beta) )

        # break
        
    TF_IDF = TF*IDF
    del TF
    # del IDF
    return TF_IDF, IDF

TF_IDF_whole_corpus, IDF_whole_corpus = compute_TF_IDF_all_train_set(hamming_vectors_2D=hamming_vectors_2D, eucledian_vectors_2D=eucledian_vectors_2D)
print(f"TF_IDF_whole_corpus.shape = {TF_IDF_whole_corpus.shape}")
print(f"IDF_whole_corpus.shape = {IDF_whole_corpus.shape}")

num_documents, num_words = (5276, 17506)
TF_IDF_whole_corpus.shape = (5276, 17506)
IDF_whole_corpus.shape = (1, 17506)


In [34]:
def form_TF_IDF_for_val_test(list_words, IDF, hamming_vectors_2D, eucledian_vectors_2D, vocab_dict, alpha=0.0001, beta=0.0001, epslion=0.000001):
    num_docs_train, num_words = eucledian_vectors_2D.shape

    TF = np.zeros((1, num_words))
    
    euc_vec = form_eucledian_vector(list_words=list_words, vocab_dict=vocab_dict)
    
    W_d_num_words_in_document = 0
    for word in list_words:
        if word in vocab_dict:
            W_d_num_words_in_document += 1

    TF = euc_vec/(W_d_num_words_in_document + epslion)

    TF_IDF = TF*IDF

    return TF_IDF

words_val = df_val['stemmed_content'].iloc[0]

TF_IDF = form_TF_IDF_for_val_test(list_words=words_val, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                    eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
print(f"TF_IDF.shape = {TF_IDF.shape}")
# print(np.unique(TF_IDF, return_counts=True))

TF_IDF.shape = (1, 17506)


In [35]:
print(df_val.columns.values)

['content' 'Label' 'stemmed_content' 'ham_vector' 'euc_vector']


# Now we start with K-Nearest Neighbor

#### Form validation 2D set

In [36]:
hamming_vectors_2D_validation = get_2D_vector(list_np_arr=df_val['ham_vector'].values)
euclidean_vectors_2D_validation = get_2D_vector(list_np_arr=df_val['euc_vector'].values)
print(hamming_vectors_2D_validation.shape, euclidean_vectors_2D_validation.shape)

(2200, 17506) (2200, 17506)


In [37]:
tf_idf_validation_set = np.asarray([
    form_TF_IDF_for_val_test(list_words=x, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
        eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001) for x in df_val['stemmed_content'].values
])
tf_idf_validation_set = tf_idf_validation_set.reshape(tf_idf_validation_set.shape[0], -1)
print(f"tf_idf_validation_set.shape = {tf_idf_validation_set.shape}")

tf_idf_validation_set.shape = (2200, 17506)


## Similarity functions for 2D vectorized

In [38]:
###### For vectorized ############
def ham(a, b):
    # return np.count_nonzero((a!=b[:, None]), axis=-1) ## returns 3-D matrix. Don't use.
    return pairwise_distances(a, b, metric='euclidean') ## since binary vectors will return hamming-distance

def euclidean(a, b):
    # return np.linalg.norm((a-b[:, None]), axis=-1)
    return pairwise_distances(a, b, metric='euclidean')


def cosine_similarity(a, b):
    # return (b.dot(a.T))/(np.linalg.norm(a, axis=1) * np.linalg.norm(b[:, None], axis=-1)) ## (a@b.T).T  ## to get r2*r1
    return pairwise_distances(a, b, metric='cosine')

In [39]:
#@title Similarity functions without pairwise-distances
####### For single loop ############
# ## Similarity functions.
# def ham(a, b):
#     return np.count_nonzero((a!=b), axis=1)

# def euclidean(a, b):
#     return np.linalg.norm((a-b), axis=1)

# ### https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
# def cosine_similarity(a, b):
#     cos_sim = np.inner(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
#     return cos_sim


# a = np.array([x for x in range(40)])
# a = np.array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1])
# a = a.reshape(5, -1)
# print("a: \n", a)

# b = np.array([1, 0, 1])
# b = b.reshape(1, -1)
# print("\nb: ", b)


# # print(ham(a, b))
# print(f"\nEuclidean(a, b) = {euclidean(a, b)}")

# a = np.array([2, 1, 3, 4, 5, 1, 10, 3, 22])
# n = 4
# indices_top = (-a).argsort()[:n]
# print(f"\n n = {n}, indices_top = {indices_top}")
# print(a[indices_top])

In [40]:
labels_train = df_train['Label'].values
print(labels_train.shape)

labels_val = df_val['Label'].values
print(labels_val.shape)

val_euc_0 = euclidean_vectors_2D_validation[0:2]
print(val_euc_0.shape)

(5276,)
(2200,)
(2, 17506)


In [41]:
### KNN Vectorized.
class KNN:
    def __init__(self, mode="hamming", to_print=False):
        self.mode = mode
        if to_print == True:
            print(f"KNN __init__(mode={mode})")
        

    def compute_distances(self, v):
        # print(f"v.shape = {v.shape}")
        if self.mode == "hamming":
            self.distances = ham(v, self.vectors_corpus)
        elif self.mode == "euclidean":
            self.distances = euclidean(v, self.vectors_corpus)
        elif self.mode == 'cosine_similarity':
            self.distances = cosine_similarity(v, self.vectors_corpus)

        # print(f"After compute_distances mode={self.mode}, distances.shape = {self.distances.shape}")
        # print(f"{self.distances}")

    def populate_vectors(self, vectors_corpus, labels):
        self.vectors_corpus = vectors_corpus
        self.labels = labels

    def predict(self, v, num_neighbors, compute_distance_flag=True):
        K = num_neighbors
        ## compute distances by using suitable similarity function.
        if compute_distance_flag == True:
            self.compute_distances(v)

        ## take argmax top results indices
        ## (-v.T).argsort(axis=0)[:K].reshape(-1, )
        indices_top = (self.distances.T).argsort(axis=0)[:K] ## highest value is negative distance ? Don't know why, +ve should be taken.

        ## apply indices to labels
        top_labels = self.labels[indices_top]

        ## take majority vote and return the label
        top_most_label = stats.mode(top_labels)[0][0]

        ## return the max label
        return top_most_label

################################################# Test #################################################

# knn = KNN(mode='hamming')
# knn.populate_vectors(hamming_vectors_2D, labels_train)
# knn.predict(val_ham_0, num_neighbors=1)

knn = KNN(mode='euclidean')
knn.populate_vectors(eucledian_vectors_2D, labels_train)
ans = knn.predict(val_euc_0, num_neighbors=1)
print(ans.shape)
print(ans)

(2,)
['Wood_Working' 'Space']


In [42]:
#@title KNN Non-Vectorized
# class KNN:
#     def __init__(self, K = 1, mode="hamming", to_print=False):
#         self.mode = mode
#         self.K = K
#         if to_print == True:
#             print(f"KNN __init__(K={K}, mode={mode})")
        

#     def populate_vectors(self, vectors_corpus, labels):
#         self.vectors_corpus = vectors_corpus
#         self.labels = labels

#     def compute_distances(self, v):
#         if self.mode == "hamming":
#             v = v.reshape(1, -1) ## Reshape vector.
#             self.distances = ham(v, self.vectors_corpus)
#         elif self.mode == "euclidean":
#             v = v.reshape(1, -1) ## Reshape vector.
#             self.distances = euclidean(v, self.vectors_corpus)
#         elif self.mode == 'cosine_similarity':
#             v = v.reshape(-1)
#             self.distances = cosine_similarity(v, self.vectors_corpus)

#         # print(f"After compute_distances mode={self.mode}, distances = {self.distances}")

#     def predict(self, v):
#         ## compute distances by using suitable similarity function.
#         self.compute_distances(v)

#         ## take argmax top results indices
#         if self.mode == 'cosine_similarity':
#             indices_top = (-self.distances).argsort()[:self.K] ## for some reason, this works for cosine-similarity
#         else:
#             indices_top = (self.distances).argsort()[:self.K] ## THIS works for hamming and euclidean

#         ## apply indices to labels
#         top_labels = self.labels[indices_top]

#         ## take majority vote and return the label
#         top_most_label = stats.mode(top_labels)[0][0]

#         ## return the max label
#         return top_most_label

# ################################################# Test #################################################

# knn = KNN(K=1, mode='hamming')
# knn.populate_vectors(hamming_vectors_2D, labels)
# knn.predict(val_ham_0)

# # knn = KNN(K=7, mode='euclidean')
# # knn.populate_vectors(eucledian_vectors_2D, labels)
# # knn.predict(val_euc_0)

## Test for one value of K

In [43]:
###### Test for one K #######
K = 1
knn = KNN(mode='hamming')
knn.populate_vectors(hamming_vectors_2D, labels_train)
y_preds = knn.predict(hamming_vectors_2D_validation, num_neighbors=K)
acc = accuracy_score(y_true=labels_val, y_pred=y_preds, normalize=True)*100
print(f"Acc Hamming K = {K} = {acc}%")

# print(f"Acc Hamming K = {K} = {np.sum(labels_val==y_preds)/len(labels_val)*100}%")
del knn


# knn = KNN(mode='euclidean')
# knn.populate_vectors(eucledian_vectors_2D, labels_train)
# y_preds = knn.predict(euclidean_vectors_2D_validation, num_neighbors=K)

# print(f"Acc Euclidean K = {K} = {np.sum(labels_val==y_preds)/len(labels_val)*100}%")


# knn = KNN(mode='cosine_similarity')
# knn.populate_vectors(TF_IDF_whole_corpus, labels_train)
# y_preds = knn.predict(tf_idf_validation_set, num_neighbors=K)

# print(f"Acc TF-IDF-Cosine K = {K} = {np.sum(labels_val==y_preds)/len(labels_val)*100}%")

Acc Hamming K = 1 = 39.68181818181818%


## Applying tests on validation set for KNN

In [44]:
%%time
## Initialize results dataframe
column_names = ["Similarity-Measure", "K", "Accuracy(%)"]
df_results_knn = pd.DataFrame(columns=column_names)

### Initialize objects
knn_ham = KNN(mode='hamming')
knn_euc = KNN(mode='euclidean')
knn_cosine = KNN(mode='cosine_similarity')

### Populate initial vectors
knn_ham.populate_vectors(hamming_vectors_2D, labels_train)
knn_euc.populate_vectors(eucledian_vectors_2D, labels_train)
knn_cosine.populate_vectors(TF_IDF_whole_corpus, labels_train)

### Compute distances for each val-set
knn_ham.compute_distances(hamming_vectors_2D_validation)
knn_euc.compute_distances(euclidean_vectors_2D_validation)
knn_cosine.compute_distances(tf_idf_validation_set)

### Predict for each value of K
for k in [1, 3, 5, 7, 9, 11, 13, 15]:
    print(f"Predicting for k = {k}")
    ## Predict and Append to dataframe.  
    ## Signature: def predict(self, v, num_neighbors, compute_distance_flag=True)
    y_preds = knn_ham.predict(hamming_vectors_2D_validation, num_neighbors=k, compute_distance_flag=False)
    acc = np.sum(labels_val==y_preds)/len(labels_val)*100 ## accuracy_score(y_true=labels_val, y_pred=y_preds, normalize=True)*100
    df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["Hamming", k, acc])
    
    y_preds = knn_euc.predict(hamming_vectors_2D_validation, num_neighbors=k, compute_distance_flag=False)
    acc = accuracy_score(y_true=labels_val, y_pred=y_preds, normalize=True)*100
    df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["Euclidean", k, acc])

    y_preds = knn_cosine.predict(hamming_vectors_2D_validation, num_neighbors=k, compute_distance_flag=False)
    acc = accuracy_score(y_true=labels_val, y_pred=y_preds, normalize=True)*100
    df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["TF-IDF-cosine-sim", k, acc])
    

### Delete each objects
del knn_ham
del knn_euc
del knn_cosine

### Print
print(f"len df_results_knn = {len(df_results_knn)}")
print(df_results_knn.head(3))

Predicting for k = 1
Predicting for k = 3
Predicting for k = 5
Predicting for k = 7
Predicting for k = 9
Predicting for k = 11
Predicting for k = 13
Predicting for k = 15
len df_results_knn = 24
  Similarity-Measure  K  Accuracy(%)
0            Hamming  1    39.681818
1          Euclidean  1    55.954545
2  TF-IDF-cosine-sim  1    78.863636
CPU times: user 2min 7s, sys: 847 ms, total: 2min 8s
Wall time: 1min 13s


In [45]:
print(f"len df_results_knn = {len(df_results_knn)}")
display(df_results_knn)
df_results_knn.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Using-2D-Pairwise/KNN-val-topics.csv', index=False)

len df_results_knn = 24


,Similarity-Measure,K,Accuracy(%)
0,Hamming,1,39.681818
1,Euclidean,1,55.954545
2,TF-IDF-cosine-sim,1,78.863636
3,Hamming,3,40.045455
4,Euclidean,3,54.590909
5,TF-IDF-cosine-sim,3,79.181818
6,Hamming,5,42.681818
7,Euclidean,5,53.590909
8,TF-IDF-cosine-sim,5,80.500000
9,Hamming,7,39.636364


In [46]:
mat = df_results_knn.values
print(mat.shape)
print(mat)

(24, 3)
[['Hamming' 1 39.68181818181818]
 ['Euclidean' 1 55.95454545454545]
 ['TF-IDF-cosine-sim' 1 78.86363636363637]
 ['Hamming' 3 40.04545454545455]
 ['Euclidean' 3 54.59090909090909]
 ['TF-IDF-cosine-sim' 3 79.18181818181819]
 ['Hamming' 5 42.68181818181818]
 ['Euclidean' 5 53.590909090909086]
 ['TF-IDF-cosine-sim' 5 80.5]
 ['Hamming' 7 39.63636363636363]
 ['Euclidean' 7 51.63636363636363]
 ['TF-IDF-cosine-sim' 7 81.54545454545455]
 ['Hamming' 9 37.59090909090909]
 ['Euclidean' 9 50.13636363636363]
 ['TF-IDF-cosine-sim' 9 82.86363636363636]
 ['Hamming' 11 35.04545454545455]
 ['Euclidean' 11 49.72727272727273]
 ['TF-IDF-cosine-sim' 11 82.5]
 ['Hamming' 13 31.772727272727273]
 ['Euclidean' 13 47.54545454545455]
 ['TF-IDF-cosine-sim' 13 83.13636363636364]
 ['Hamming' 15 30.045454545454547]
 ['Euclidean' 15 45.18181818181819]
 ['TF-IDF-cosine-sim' 15 83.31818181818181]]


## Naive Bayes

### Combine all documents of each class i into one document

In [47]:
# unique_labels = np.unique(df_train['Label'].values)
# dictionary_list_words_for_NB = {}
# # for label in unique_labels:
# #     if label not in dictionary_list_words:
# #         dictionary_list_words_for_NB[label] = [] ## create new list.
    
# for (list_words, label) in zip(df_train['stemmed_content'].values, df_train['Label'].values):
#     if label not in dictionary_list_words_for_NB:
#         dictionary_list_words_for_NB[label] = [] ## create new list.
#     dictionary_list_words_for_NB[label].append(list_words)

# ## reduce/flat-out to make 1D list.
# ## https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
# for label in dictionary_list_words_for_NB:
#     dictionary_list_words_for_NB[label] = reduce(operator.concat, dictionary_list_words_for_NB[label])

In [48]:
## https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
def keywithmaxval(d):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

In [49]:
class NaiveBayes:
    def __init__(self, vocab_size, alpha=0.01, to_print=False):
        if to_print == True:
            print(f"NaiveBayes __init(alpha={alpha})__")
        self.alpha = alpha
        self.vocab_size = vocab_size
        self.dictionary_list_words_for_NB = {}
        self.dictionary_prior_probabilities = {}
        self.dictionary_count_words_per_class = {}
        self.dictionary_total_words_per_class = {}
    

    def fit(self, list_list_words, labels):
        self.list_list_words = list_list_words
        self.labels = labels
        self.form_dictionary_list_words()
        self.compute_probabilities()


    def form_dictionary_list_words(self):
        for (list_words, label) in zip(self.list_list_words, self.labels):
            if label not in self.dictionary_list_words_for_NB:
                self.dictionary_list_words_for_NB[label] = [] ## create new list.
            self.dictionary_list_words_for_NB[label].append(list_words)
        
        for label in self.dictionary_list_words_for_NB: ## reduce/flat-out to make 1D list.
            self.dictionary_list_words_for_NB[label] = reduce(operator.concat, self.dictionary_list_words_for_NB[label])


    def compute_probabilities(self):
        ## Compute prior probabilities/initial guesses
        (classes, cnts) = np.unique(self.labels, return_counts=True)
        cnts = cnts/np.sum(cnts) ## C_i / (C_1 + C_2 + ... + C_n)
        for (lab, itr) in zip(classes, range(len(classes))):
            self.dictionary_prior_probabilities[lab] = cnts[itr]

        ## Compute per-word probabilities

        ## Counter increment
        for lab in classes:
            num_words_this_class = 0
            self.dictionary_count_words_per_class[lab] = {}
            for word in self.dictionary_list_words_for_NB[lab]:
                if word not in self.dictionary_count_words_per_class[lab]:
                    self.dictionary_count_words_per_class[lab][word] = 0 ## initialize counter to 0.
                self.dictionary_count_words_per_class[lab][word] = self.dictionary_count_words_per_class[lab][word] + 1 ## increment counter
                num_words_this_class += 1
            self.dictionary_total_words_per_class[lab] = num_words_this_class

    def predict(self, list_words):
        ## Compute probabilities for each label.
        dict_probabilities_per_class = {}

        for lab in np.unique(self.labels):
            prob_log_curr_class = np.log(self.dictionary_prior_probabilities[lab]) ## start with prior probabilities
            # prob_log_curr_class = (self.dictionary_prior_probabilities[lab]) ## start with prior probabilities
            # print(f"Before, prob_log_curr_class = {prob_log_curr_class}")
            for word in list_words: ## iterate per word
                if word in self.dictionary_count_words_per_class[lab]: ## if word exists in THIS document.
                    ## use smoothing factor alpha
                    # prob_log_curr_class += np.log((self.dictionary_count_words_per_class[lab][word] + self.alpha)/(self.dictionary_total_words_per_class[lab] + self.alpha*self.vocab_size)) 
                    word_prob = self.dictionary_count_words_per_class[lab][word]
                    # prob_log_curr_class = prob_log_curr_class*word_prob
                else:
                    word_prob = 0
                prob_log_curr_class += np.log( (word_prob + self.alpha)/(self.dictionary_total_words_per_class[lab] + self.alpha*self.vocab_size) )


            dict_probabilities_per_class[lab] = prob_log_curr_class
            # print(dict_probabilities_per_class)

        ## Get max probability class.
        ## https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
        return keywithmaxval(d=dict_probabilities_per_class)




######################################### Checking ############################################################
NB = NaiveBayes(alpha=0.01, vocab_size=len(dictionary_vocab))
NB.fit(list_list_words=df_train['stemmed_content'].values, labels=df_train['Label'].values)

check = df_val['stemmed_content'].iloc[0]
p = NB.predict(check)
print(p)

Wood_Working


In [50]:
list_new = [
['coffee', 'tea', 'dew', 'dew', 'dew', 'dew'],
['coffee', 'noir', 'homelander', 'dew'],
['noir', 'noir', 'fool', 'fool', 'noir']
]

labs_new = [
    'bev',
    'supe',
    'misc'
]

vocab_size = 6

NB = NaiveBayes(alpha=0.01, vocab_size=6)
NB.fit(list_new, labs_new)
print(NB.predict(['coffee', 'tea', 'dew', 'dew', 'dew', 'dew']))

bev


## Validation on NaiveBayes

In [51]:
column_names = ["ALPHA", "Accuracy(%)"]

df_results_NB = pd.DataFrame(columns=column_names)

display(df_results_NB.head())

,ALPHA,Accuracy(%)


In [52]:
alpha_values = np.linspace(start=0.01, stop=1.0, num=100)
print(alpha_values)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99 1.  ]


In [53]:
### Validation NaiveBayes ###
for alpha in (alpha_values):
    NB = NaiveBayes(alpha=alpha, vocab_size=len(dictionary_vocab), to_print=True)
    NB.fit(list_list_words=df_train['stemmed_content'].values, labels=df_train['Label'].values)

    nb_acc = 0
    ## Predict each val set ##
    for (x, y) in (zip(df_val['stemmed_content'].values, df_val['Label'].values)):
        y_pred = NB.predict(x)
        if y_pred == y:
            nb_acc += 1


    ## Append to dataframe and print.
    # print(f"NB alpha = {alpha}, accuracy = {nb_acc/len(df_val)*100} %")
    df_results_NB = add_to_dataframe(df_old=df_results_NB, to_add=[alpha, nb_acc/len(df_val)*100])
    del NB

NaiveBayes __init(alpha=0.01)__
NaiveBayes __init(alpha=0.02)__
NaiveBayes __init(alpha=0.03)__
NaiveBayes __init(alpha=0.05)__
NaiveBayes __init(alpha=0.060000000000000005)__
NaiveBayes __init(alpha=0.06999999999999999)__
NaiveBayes __init(alpha=0.09)__
NaiveBayes __init(alpha=0.09999999999999999)__
NaiveBayes __init(alpha=0.11)__
NaiveBayes __init(alpha=0.12)__
NaiveBayes __init(alpha=0.13)__
NaiveBayes __init(alpha=0.14)__
NaiveBayes __init(alpha=0.15000000000000002)__
NaiveBayes __init(alpha=0.16)__
NaiveBayes __init(alpha=0.17)__
NaiveBayes __init(alpha=0.18000000000000002)__
NaiveBayes __init(alpha=0.19)__
NaiveBayes __init(alpha=0.2)__
NaiveBayes __init(alpha=0.21000000000000002)__
NaiveBayes __init(alpha=0.22)__
NaiveBayes __init(alpha=0.23)__
NaiveBayes __init(alpha=0.24000000000000002)__
NaiveBayes __init(alpha=0.25)__
NaiveBayes __init(alpha=0.26)__
NaiveBayes __init(alpha=0.27)__
NaiveBayes __init(alpha=0.28)__
NaiveBayes __init(alpha=0.29000000000000004)__
NaiveBayes __ini

In [54]:
df_results_NB.sort_values(by=['Accuracy(%)'], inplace=True, ascending=False)
display(df_results_NB.head(10))
df_results_NB.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Using-2D-Pairwise/NB-val-topics.csv', index=False)

,ALPHA,Accuracy(%)
3,0.04,88.500000
1,0.02,88.500000
2,0.03,88.454545
4,0.05,88.454545
6,0.07,88.454545
0,0.01,88.409091
14,0.15,88.409091
23,0.24,88.409091
17,0.18,88.409091
16,0.17,88.409091


In [55]:
display(df_results_NB.head(10))

,ALPHA,Accuracy(%)
3,0.04,88.500000
1,0.02,88.500000
2,0.03,88.454545
4,0.05,88.454545
6,0.07,88.454545
0,0.01,88.409091
14,0.15,88.409091
23,0.24,88.409091
17,0.18,88.409091
16,0.17,88.409091


# Hypothesis testing on Test Dataset

In [57]:
del hamming_vectors_2D_validation
del euclidean_vectors_2D_validation
del tf_idf_validation_set

### Create and fit best performing models on validation set

In [58]:
words_val = df_val['stemmed_content'].iloc[0]
print(words_val)

['sagul', 'give', 'deflect', 'valu', 'horizont', 'shelf', 'span', 'use', 'variou', 'materi', 'thick', 'unless', 'work', 'extrem', 'heavi', 'load', 'weight', 'countertop', 'peopl', 'danc', 'materi', 'vertic', 'cabinet', 'wall', 'adequ', 'make', 'wall', 'thinner', 'still', 'remain', 'surprisingli', 'strong', 'long', 'cabinet', 'design', 'prevent', 'rack', 'even', 'particleboard', 'known', 'strength', 'hold', 'well', 'vertic', 'compressionbuckl', 'may', 'know', 'youv', 'ever', 'pack', 'book', 'store', 'cheap', 'particleboard', 'bookcas']


In [75]:
##### KNN was K=15, TF-IDF #####
K_best = 15
knn = KNN(mode='cosine_similarity', to_print=True)
knn.populate_vectors(TF_IDF_whole_corpus, labels_train)

In [62]:
val_tf_idf = form_TF_IDF_for_val_test(list_words=words_val, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
print(knn.predict(val_tf_idf, num_neighbors=15))

['Wood_Working']


In [63]:
##### Naive Bayes was alpha = 0.02/0.04, we will take 0.04 #####
NB = NaiveBayes(alpha=0.04, vocab_size=len(dictionary_vocab), to_print=True)
NB.fit(list_list_words=df_train['stemmed_content'].values, labels=df_train['Label'].values)

NaiveBayes __init(alpha=0.04)__


In [64]:
print(NB.predict(words_val))

Wood_Working


In [65]:
del df_train
del df_val

### Split test dataset 50 iterations per 10 of each topic

In [66]:
print(df_test.columns.values)

['content' 'Label' 'stemmed_content' 'ham_vector' 'euc_vector']


In [67]:
df_test.sort_values(by=['Label'], inplace=True)
df_test.drop(labels=['content', 'ham_vector', 'euc_vector'], axis=1, inplace=True)

In [77]:
column_names = ["KNN-Acc(%)", "NB-Acc(%)"]

df_results_test_set = pd.DataFrame(columns=column_names)

print(df_results_test_set.head())

Empty DataFrame
Columns: [KNN-Acc(%), NB-Acc(%)]
Index: []


In [78]:
num_docs_in_each_topic = 500 ##
initial_offset = np.array([i*num_docs_in_each_topic for i in range(0, 11)])
for counter_test in range(0, 50): ## run iterations 50 times
    offsets = initial_offset + counter_test*10
    start_indices = offsets
    end_indices = offsets + 10
    
    print("\nCounter = ", counter_test)
    # print(offsets)
    # print(start_indices)
    # print(end_indices)

    ### Testing here ###
    nb_correct = knn_correct = 0
    for i in range(len(start_indices)): ## add all to list.
        # print(np.unique(df_test.iloc[start_indices[i]:end_indices[i]]['Label'].values, return_counts=True), end=' ')
        for (x, y) in zip(df_test.iloc[start_indices[i]:end_indices[i]]['stemmed_content'].values, df_test.iloc[start_indices[i]:end_indices[i]]['Label'].values):
            
            #### For KNN ####
            x_tf_idf = form_TF_IDF_for_val_test(list_words=x, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
            y_pred = knn.predict(x_tf_idf, num_neighbors=K_best)
            if y_pred == y:
                knn_correct += 1
            
            #### For Naive Bayes #####
            y_pred = NB.predict(x)
            if y_pred == y:
                nb_correct += 1
            
    
    print(f"Done for counter_test = {counter_test}")

    NUM_DOCUMENTS = 10* len(np.unique(df_test['Label'].values))
    knn_acc = knn_correct/( NUM_DOCUMENTS )*100  ## 10*11 total 110 test documents per iteration. [50 iterations]
    nb_acc = nb_correct/( NUM_DOCUMENTS )*100
    print(f"KNN-Acc = {knn_acc}%, NB-Acc = {nb_acc}%")

    df_results_test_set = add_to_dataframe(df_old=df_results_test_set, to_add=[knn_acc, nb_acc])


Counter =  0
Done for counter_test = 0
KNN-Acc = 85.45454545454545%, NB-Acc = 88.18181818181819%

Counter =  1
Done for counter_test = 1
KNN-Acc = 80.9090909090909%, NB-Acc = 84.54545454545455%

Counter =  2
Done for counter_test = 2
KNN-Acc = 79.0909090909091%, NB-Acc = 90.9090909090909%

Counter =  3
Done for counter_test = 3
KNN-Acc = 87.27272727272727%, NB-Acc = 90.0%

Counter =  4
Done for counter_test = 4
KNN-Acc = 89.0909090909091%, NB-Acc = 91.81818181818183%

Counter =  5
Done for counter_test = 5
KNN-Acc = 83.63636363636363%, NB-Acc = 86.36363636363636%

Counter =  6
Done for counter_test = 6
KNN-Acc = 77.27272727272727%, NB-Acc = 90.9090909090909%

Counter =  7
Done for counter_test = 7
KNN-Acc = 83.63636363636363%, NB-Acc = 91.81818181818183%

Counter =  8
Done for counter_test = 8
KNN-Acc = 83.63636363636363%, NB-Acc = 90.9090909090909%

Counter =  9
Done for counter_test = 9
KNN-Acc = 80.9090909090909%, NB-Acc = 89.0909090909091%

Counter =  10
Done for counter_test = 10

In [80]:
df_results_test_set.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Using-2D-Pairwise/Test-Set-11-KNN-NB.csv', index=False)

In [79]:
print(f"len df_results_test_set = {len(df_results_test_set)}")
display(df_results_test_set)

len df_results_test_set = 50


,KNN-Acc(%),NB-Acc(%)
0,85.454545,88.181818
1,80.909091,84.545455
2,79.090909,90.909091
3,87.272727,90.000000
4,89.090909,91.818182
5,83.636364,86.363636
6,77.272727,90.909091
7,83.636364,91.818182
8,83.636364,90.909091
9,80.909091,89.090909


## Load and analyze.

In [81]:
# df_results_test_set = pd.read_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Test-Set-KNN-NB.csv')
print(df_results_test_set.head(5))

   KNN-Acc(%)  NB-Acc(%)
0   85.454545  88.181818
1   80.909091  84.545455
2   79.090909  90.909091
3   87.272727  90.000000
4   89.090909  91.818182


In [82]:
knn_acc = df_results_test_set['KNN-Acc(%)'].values
nb_acc = df_results_test_set['NB-Acc(%)'].values
print(knn_acc.shape, nb_acc.shape)

(50,) (50,)


In [92]:
column_names = ["Method", "Mean-Acc(%)", "Std-dev", "Std-Error", "Minimum-Acc(%)", "Maximum-Acc(%)"]

df_stats = pd.DataFrame(columns=column_names)

print(df_stats.head())

Empty DataFrame
Columns: [Method, Mean-Acc(%), Std-dev, Std-Error, Minimum-Acc(%), Maximum-Acc(%)]
Index: []


In [93]:
## Summarized results.
df_stats = add_to_dataframe(df_old=df_stats, to_add=["KNN K=15, TF-IDF", np.mean(knn_acc), np.std(knn_acc), stats.sem(knn_acc, axis=None, ddof=0), min(knn_acc), max(knn_acc)])
df_stats = add_to_dataframe(df_old=df_stats, to_add=["NB alpha = 0.04", np.mean(nb_acc), np.std(nb_acc), stats.sem(nb_acc, axis=None, ddof=0), min(nb_acc), max(nb_acc)])

In [94]:
print(df_stats)

             Method  Mean-Acc(%)  ...  Minimum-Acc(%)  Maximum-Acc(%)
0  KNN K=15, TF-IDF    83.381818  ...       70.909091       89.090909
1   NB alpha = 0.04    89.763636  ...       82.727273       95.454545

[2 rows x 6 columns]


## Computing T-statistics

In [95]:
knn_acc = df_results_test_set['KNN-Acc(%)'].values
nb_acc = df_results_test_set['NB-Acc(%)'].values

In [97]:
## https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

# ans = stats.ttest_rel(nb_acc, knn_acc)
# ans = stats.ttest_ind(nb_acc, knn_acc, equal_var=True)
ans = stats.ttest_rel(nb_acc, knn_acc)
print(ans)

Ttest_relResult(statistic=14.245213763349957, pvalue=4.766189926008192e-19)
